In [31]:
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
df = pd.read_csv("Data/kitCapstoneJan24_data.csv", sep=";")

## selecting the country that we want

In [33]:
selected_country = 'Country A'
#Get rows with for country A

df_country_a = df[df['countryname'] == selected_country]
df_country_a

,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,sales_actuals_monthly__vehiclegroup04__retail,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,357,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,358,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,359,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## building a additional feature for months (1-12)

important for meanbymonth

In [34]:
df_country_a['month_normalized'] = df_country_a['month'] % 12
df_country_a

df_country_a = df_country_a[['month_normalized','month', 'countryname'] + [col for col in df_country_a.columns if col not in ['month_normalized','month', 'countryname']]]

df_country_a


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,1,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,8,356,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,9,357,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,10,358,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,11,359,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## select an vehiclegroup

In [35]:
selcted_vehiclegroup = 'vehiclegroup01'

# delete all columns that are for a diffe

df1 = df_country_a

# create new dataframe with month and all columns containing vehiclegroup01
df2 = df1.filter(regex=selcted_vehiclegroup)
df2.head()

# drop columns in df1 which contain vehiclegroup
df3 = df1.drop(df1.filter(regex='vehiclegroup').columns, axis=1)
df3.head()

# merge df2 and df3 by index
df4 = df2.merge(df3, left_index=True, right_index=True)
df4.head()

# is there a column month in df4?
df4.columns.str.contains('month').any()
# column month should be the first column, second column is countryname
df4 = df4[['month_normalized','month', 'countryname'] + [col for col in df4.columns if col not in ['month_normalized','month', 'countryname']]]

In [36]:
# drop rows after row 258
test = df4.drop(df4.index[258:])

## build different dataframes for internal and external data

In [37]:
# dataframe with first 18 columns (internal data)
df_internal = test.iloc[:, :21]
df_internal.head()

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_flow_monthly__vehiclegroup01__cp8,sales_stock_monthly__vehiclegroup01__physical,sales_stock_monthly__vehiclegroup01__grossall,sales_stock_monthly__vehiclegroup01__netimporteranddealer,customer_vehicle_relationships__vehiclegroup01__terminated_cars,...,leads__vehiclegroup01__won,leads__vehiclegroup01__first_touch_point,leads__vehiclegroup01__lost,leasing_contracts__vehiclegroup01__end,leasing_contracts__vehiclegroup01__start,premises__vehiclegroup01__flag_order_start,premises__vehiclegroup01__in_sales,premises__vehiclegroup01__flag_production_start,premises__vehiclegroup01__in_production,quota__vehiclegroup01__aq
0,1,1,Country A,296.0,352.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,0.0,81.0,NaN,NaN,NaN,NaN,NaN
1,2,2,Country A,414.0,372.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,4.0,130.0,NaN,NaN,NaN,NaN,NaN
2,3,3,Country A,405.0,604.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,2.0,145.0,NaN,NaN,NaN,NaN,NaN
3,4,4,Country A,359.0,483.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,2.0,183.0,NaN,NaN,NaN,NaN,NaN
4,5,5,Country A,403.0,474.0,NaN,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,4.0,201.0,NaN,NaN,NaN,NaN,NaN


# dataframe with external features
df_external = test.iloc[:, 21:]
df_external = test['']df_external.
df_external.head()

In [38]:
# dataframe with external features
df_external = test.iloc[:, 21:]
df_external.loc[:, 'month_normalized'] = test['month_normalized']
df_external.head()

,global_consumer__alcoholic_beverages,global_consumer__apparel,global_consumer__apparel_footwear,global_consumer__appliances,global_consumer__automobiles,global_consumer__average_disposable_income_of_urban_households_nominal,global_consumer__average_disposable_income_of_urban_households_real,global_consumer__average_income,global_consumer__beer,global_consumer__building_materials_garden_supply,...,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi,month_normalized
0,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,46.394457,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


# building target values


In [39]:
#take every row from 50 to 255 and 
#X = df_internal.iloc[49:255,:-3]
X = df_internal.iloc[49:255, ~df_internal.columns.str.contains('sales_actuals_monthly__vehiclegroup01__orderintake')]
y = df_internal.iloc[49:255, 3]

X_with_target = df_internal.iloc[49:255,:]


In [40]:
X_with_target

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_flow_monthly__vehiclegroup01__cp8,sales_stock_monthly__vehiclegroup01__physical,sales_stock_monthly__vehiclegroup01__grossall,sales_stock_monthly__vehiclegroup01__netimporteranddealer,customer_vehicle_relationships__vehiclegroup01__terminated_cars,...,leads__vehiclegroup01__won,leads__vehiclegroup01__first_touch_point,leads__vehiclegroup01__lost,leasing_contracts__vehiclegroup01__end,leasing_contracts__vehiclegroup01__start,premises__vehiclegroup01__flag_order_start,premises__vehiclegroup01__in_sales,premises__vehiclegroup01__flag_production_start,premises__vehiclegroup01__in_production,quota__vehiclegroup01__aq
49,2,50,Country A,215.0,256.0,NaN,NaN,NaN,NaN,78.0,...,NaN,NaN,NaN,296.0,197.0,0.0,0.0,0.0,0.0,NaN
50,3,51,Country A,250.0,488.0,NaN,NaN,NaN,NaN,79.0,...,NaN,NaN,NaN,376.0,285.0,0.0,0.0,0.0,0.0,NaN
51,4,52,Country A,222.0,552.0,NaN,NaN,NaN,NaN,84.0,...,NaN,NaN,NaN,374.0,346.0,0.0,0.0,200.0,51.0,NaN
52,5,53,Country A,179.0,411.0,NaN,NaN,NaN,NaN,73.0,...,NaN,NaN,NaN,381.0,401.0,0.0,0.0,0.0,51.0,NaN
53,6,54,Country A,209.0,366.0,NaN,NaN,NaN,NaN,143.0,...,NaN,NaN,NaN,313.0,328.0,0.0,0.0,0.0,51.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,11,251,Country A,323.0,355.0,317.0,331.0,311.0,138.0,564.0,...,615.0,218.0,241.0,590.0,432.0,0.0,417.0,0.0,385.0,326.0
251,0,252,Country A,262.0,579.0,305.0,259.0,214.0,180.0,355.0,...,528.0,184.0,195.0,478.0,299.0,0.0,417.0,0.0,385.0,279.0
252,1,253,Country A,266.0,195.0,179.0,272.0,249.0,181.0,353.0,...,445.0,187.0,213.0,279.0,128.0,0.0,417.0,0.0,385.0,138.0
253,2,254,Country A,531.0,247.0,297.0,326.0,357.0,173.0,371.0,...,757.0,267.0,367.0,319.0,158.0,333.0,472.0,0.0,385.0,238.0


## testing wheel

In [65]:
from scipy.interpolate import interp1d



use_all_features = True
# if false, assign checkup_features here
checkup_features = ['quota__vehiclegroup01__aq'],['sales_flow_monthly__vehiclegroup01__cp8']
input_df = df_internal

#methods
mean_per_month = True
median_per_month = True
interpolation_linear = True
interpolation_spline = True
fill_zero = True

if use_all_features:
    checkup_features = input_df.columns
for feature in checkup_features:
   # print(feature)
    if input_df[feature].isna().any():
        if mean_per_month:
            # add mean per month
            # Fill in missing values using the corresponding month values
            month_means = {}
            for i in range(len(input_df)):
                month = input_df.iloc[i]['month_normalized']
                month_values = input_df.loc[input_df['month_normalized'] == month, feature]
                if not month_values.isnull().values.all():
                    month_means[month] = month_values.mean()

            # Fill in missing values using the corresponding month values
            input_df[feature+'_meanbymonth']=input_df[feature].loc[:].fillna(input_df['month_normalized'].map(month_means))

        if median_per_month:
            # add median per month
            # Fill in missing values using the corresponding month values
            month_medians = {}
            for i in range(len(input_df)):
                month = input_df.iloc[i]['month_normalized']
                month_values = input_df.loc[input_df['month_normalized'] == month, feature]
                if not month_values.isnull().values.all():
                    month_medians[month] = month_values.median(skipna=True)
                    print(month_medians[month])
            
            input_df[feature + '_medianbymonth'] = (input_df[feature].loc[:].
                                                    fillna(input_df['month_normalized']
                                                           .map(month_medians)))

        if interpolation_linear:
            input_df[feature+'_interpolated_linear']=input_df[feature].interpolate(method='linear', limit_direction = 'both')

        if interpolation_spline:
            input[feature + 'interpolation_spline'] = interp1d(input_df[feature], kind='cubic', fill_value='extrapolate')

    
        if fill_zero:
            input_df[feature+'_fill_zero']=input_df[feature].fillna(0)

        input_df.drop(columns=feature, inplace=True)
       
output_df = input_df


183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0
262.0
320.0
320.5
264.0
327.0
293.0
17.0
279.0
249.0
317.0
252.0
183.0


In [68]:
output_df.head(15)

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,customer_vehicle_relationships__vehiclegroup01__terminated_cars,customer_vehicle_relationships__vehiclegroup01__active_cars,leasing_contracts__vehiclegroup01__end,leasing_contracts__vehiclegroup01__start,sales_flow_monthly__vehiclegroup01__cp8_meanbymonth,...,premises__vehiclegroup01__flag_production_start_interpolated_linear,premises__vehiclegroup01__flag_production_start_fill_zero,premises__vehiclegroup01__in_production_meanbymonth,premises__vehiclegroup01__in_production_medianbymonth,premises__vehiclegroup01__in_production_interpolated_linear,premises__vehiclegroup01__in_production_fill_zero,quota__vehiclegroup01__aq_meanbymonth,quota__vehiclegroup01__aq_medianbymonth,quota__vehiclegroup01__aq_interpolated_linear,quota__vehiclegroup01__aq_fill_zero
0,1,1,Country A,296.0,352.0,0.0,25.0,0.0,81.0,175.888889,...,0.0,0.0,351.888889,372.0,0.0,0.0,182.562500,174.5,83.0,0.0
1,2,2,Country A,414.0,372.0,1.0,102.0,4.0,130.0,245.888889,...,0.0,0.0,363.222222,385.0,0.0,0.0,287.375000,283.5,83.0,0.0
2,3,3,Country A,405.0,604.0,2.0,152.0,2.0,145.0,273.800000,...,0.0,0.0,356.000000,372.0,0.0,0.0,345.000000,348.0,83.0,0.0
3,4,4,Country A,359.0,483.0,2.0,75.0,2.0,183.0,270.100000,...,0.0,0.0,374.555556,397.5,0.0,0.0,329.437500,328.5,83.0,0.0
4,5,5,Country A,403.0,474.0,10.0,91.0,4.0,201.0,268.400000,...,0.0,0.0,393.055556,410.0,0.0,0.0,343.500000,298.0,83.0,0.0
5,6,6,Country A,196.0,456.0,2.0,55.0,2.0,156.0,352.800000,...,0.0,0.0,404.500000,410.0,0.0,0.0,370.187500,335.5,83.0,0.0
6,7,7,Country A,387.0,533.0,7.0,166.0,4.0,215.0,268.222222,...,0.0,0.0,395.294118,385.0,0.0,0.0,378.933333,384.0,83.0,0.0
7,8,8,Country A,281.0,224.0,10.0,189.0,13.0,163.0,24.666667,...,0.0,0.0,331.944444,346.0,0.0,0.0,73.714286,81.0,83.0,0.0
8,9,9,Country A,246.0,309.0,19.0,194.0,36.0,78.0,226.000000,...,0.0,0.0,361.833333,359.0,0.0,0.0,319.733333,310.0,83.0,0.0
9,10,10,Country A,290.0,365.0,14.0,159.0,13.0,131.0,240.777778,...,0.0,0.0,378.888889,372.0,0.0,0.0,327.437500,299.5,83.0,0.0


## SHAP

In [ ]:
import xgboost
import shap
from sklearn import metrics
import matplotlib
shap.initjs()

In [ ]:
X= output_df
y= df_internal.iloc[:, 3]

y.head()

In [ ]:
model = xgboost.XGBRegressor().fit(X, y)

print('Accuracy Score of the Model is: ', model.score(X, y))

explainer = shap.Explainer(model)
shap_values = explainer(X)

In [ ]:
shap.summary_plot(shap_values, X)
shap.summary_plot(shap_values, X, plot_type='bar', max_display=10)

In [ ]:
summaryPlot_data = shap.summary_plot(shap_values, X)

feature_names = summaryPlot_data.data[0]
importance_values = summaryPlot_data.data[1]

# Get the top feature

top_feature_index = importance_values.idxmax()
top_feature_name = feature_names[top_feature_index]

In [ ]:
excel_file_path = "shap_values_output.xlsx"
shap_df = pd.DataFrame(shap_values, columns=X.columns)

# Export the DataFrame to an Excel file
shap_df.to_excel(excel_file_path, index=False)

print(f"SHAP values exported to {excel_file_path}")

In [ ]:
# Excel output top 25
# Influence on final value